In [ ]:
#r "nuget:NewtonSoft.Json,12.0.3"
#r "E:\Projects\Horizon Zero Dawn\hzd_test\HZDCoreEditor\bin\Release\netstandard2.1\win-x64\HZDCoreEditor.dll"

using System;
using System.IO;
using HZDCoreEditor;
using Newtonsoft.Json;

// Absolute path to Horizon Zero Dawn bin files
//static readonly string HZDRootPath = @"C:\Program Files (x86)\Steam\steamapps\common\Horizon Zero Dawn\Packed_DX12";

// Absolute path to Death Stranding bin files
//static readonly string DSRootPath = @"C:\Program Files (x86)\Steam\steamapps\common\Death Stranding\data";

// Absolute path to a HZDCoreTools.exe build
static readonly string CoreToolsExecutablePath = Path.Combine(Environment.CurrentDirectory, @"..\HZDCoreTools\bin\Release\net6.0-windows\HZDCoreTools.exe");

### Extract all HZD core files (excluding streams)

In [ ]:
#!powershell
#!share --from csharp CoreToolsExecutablePath

$commandArgs = @(
    'unpack',
    '-i', 'C:\Program Files (x86)\Steam\steamapps\common\Horizon Zero Dawn\Packed_DX12\*.bin',
    '-o', 'C:\Program Files (x86)\Steam\steamapps\common\Horizon Zero Dawn\Packed_DX12\extracted\',
    'r', '\.stream'
)

& $CoreToolsExecutablePath $commandArgs

### Rebuild index files for both games

Extract all strings contained in the archives

In [ ]:
#!powershell
#!share --from csharp CoreToolsExecutablePath

# Extract HZD strings
$commandArgs = @(
    '--horizonzerodawn',
    'exportstrings',
    '-i', 'C:\Program Files (x86)\Steam\steamapps\common\Horizon Zero Dawn\Packed_DX12\*.bin',
    '-o', 'dumped_hzd_paths.txt',
    '--validpathsonly'
)

Write-Host "Run Horizon Zero Dawn"
(& $CoreToolsExecutablePath $commandArgs) | Select-String -Pattern "^((?!Mounting).)*$"

# Extract DS strings
$commandArgs = @(
    '--deathstranding',
    'exportstrings',
    '-i', 'C:\Program Files (x86)\Steam\steamapps\common\Death Stranding\data\*.bin',
    '-o', 'dumped_ds_paths.txt',
    '--validpathsonly'
)

Write-Host "Run Death Stranding"
(& $CoreToolsExecutablePath $commandArgs) | Select-String -Pattern "^((?!Mounting).)*$"

Combine all string lists (missing files are ignored, duplicates are ignored)

In [ ]:
#!powershell

Get-Content All_HZD_Valid_Core_Paths.txt, All_DS_Valid_Core_Paths.txt, dumped_hzd_paths.txt, dumped_ds_paths.txt | Set-Content amalgamation.txt

Create the new index files

In [ ]:
#!powershell
#!share --from csharp CoreToolsExecutablePath

# Rebuild HZD index files
$commandArgs = @(
    '--horizonzerodawn',
    'rebuildindexfiles',
    '-i', 'C:\Program Files (x86)\Steam\steamapps\common\Horizon Zero Dawn\Packed_DX12\*.bin',
    '-o', 'C:\Program Files (x86)\Steam\steamapps\common\Horizon Zero Dawn\Packed_DX12\*.idx',
    '-l', 'amalgamation.txt'
)

Write-Host "Run Horizon Zero Dawn"
(& $CoreToolsExecutablePath $commandArgs) | Select-String -Pattern "^((?!Processing).)*$"

# Rebuild DS index files
$commandArgs = @(
    '--deathstranding',
    'rebuildindexfiles',
    '-i', 'C:\Program Files (x86)\Steam\steamapps\common\Death Stranding\data\*.bin',
    '-o', 'C:\Program Files (x86)\Steam\steamapps\common\Death Stranding\data\*.idx',
    '-l', 'amalgamation.txt',
    '--skipmissing'
)

Write-Host "Run Death Stranding"
(& $CoreToolsExecutablePath $commandArgs) | Select-String -Pattern "^((?!Processing).)*$"

### Convert a core file to json in C#

In [ ]:
var core = Decima.CoreBinary.FromFile(@"C:\Program Files (x86)\Steam\steamapps\common\Horizon Zero Dawn\Packed_DX12\extracted\default_assets\photomode\photomode.core");
var json = JsonConvert.SerializeObject(core, new JsonSerializerSettings()
{
    Formatting = Formatting.Indented,
    TypeNameHandling = TypeNameHandling.Objects,/* TypeNameHandling.None */
    ReferenceLoopHandling = ReferenceLoopHandling.Ignore,
});

Console.Write(json);

### List all possible languages in Horizon Zero Dawn and Death Stranding

In [ ]:
var possibleLanguages = Enum.GetNames(typeof(Decima.DS.ELanguage))
                .Concat(Enum.GetNames(typeof(Decima.HZD.ELanguage)))
                .Select(x => x.ToLower())
                .Distinct()
                .ToArray();

Console.WriteLine("All possible languages: ");

foreach (string lang in possibleLanguages)
    Console.Write($"{lang} ");

Console.WriteLine();

### Map GraphNodeProgram entry point names to their addresses in the DS executable

Export names

In [ ]:
#!powershell
#!share --from csharp CoreToolsExecutablePath

$commandArgs = @(
    '--deathstranding',
    'exportentrypointnames',
    '-i', 'C:\Program Files (x86)\Steam\steamapps\common\Death Stranding\data\*.bin',
    '-o', 'dumped_ds_entrypoints.csv'
)

(& $CoreToolsExecutablePath $commandArgs) | Select-String -Pattern "^((?!Mounting).)*$"

Generate IDC script

In [ ]:
var inputEntrypointNames = File.ReadAllLines("dumped_ds_entrypoints.csv");
var inputAddresses = File.ReadAllLines(@"C:\ds_rtti_export\entrypoint_crc_listing.csv");

var crcToNames = new Dictionary<ulong, string>();
var mappedLines = new List<string>();

// CSV -> dictionary
foreach (string[] tokens in inputEntrypointNames.Select(x => x.Split(',')))
{
    crcToNames.TryAdd(ulong.Parse(tokens[0].Substring(2), System.Globalization.NumberStyles.HexNumber), tokens[1]);
}

// Combine the address with the { crc, name } map
mappedLines.Add("#include <idc.idc>\n\nstatic main()\n{");
foreach (string[] tokens in inputAddresses.Select(x => x.Split(',')))
{
    var addr = ulong.Parse(tokens[0].Substring(2), System.Globalization.NumberStyles.HexNumber);
    var crc = ulong.Parse(tokens[1].Substring(2), System.Globalization.NumberStyles.HexNumber);

    if (crcToNames.TryGetValue(crc, out string name))
        mappedLines.Add($"set_name(0x{addr:X}, \"{name}\", SN_FORCE|SN_DELTAIL|SN_NOWARN);// 0x{crc:X}");
    else
        mappedLines.Add($"set_name(0x{addr:X}, \"EntryPoint_UNKNOWN_CRC_0x{crc:X}\", SN_FORCE|SN_DELTAIL|SN_NOWARN);// 0x{crc:X}");
}
mappedLines.Add("}");

File.WriteAllLines("ida_dumped_ds_entrypoints.idc", mappedLines);